## Test Real Entities

In [1]:
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook(db_name='rentitynb')
tlnb.print_info()


INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running stamp_revision  -> 48dd68d06c60


Timelink version: 1.1.15
Project name: test-project
Project home: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/test-project
Database type: sqlite
Database name: rentitynb
Kleio image: timelinkserver/kleio-server
Kleio server token: pL8fs...
Kleio server URL: http://127.0.0.1:8089
Kleio server home: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/test-project
Kleio server container: condescending_lewin
Kleio version requested: latest
Kleio server version: 12.6.577 (2024-10-24 16:53:53)
SQLite directory: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/test-project/database/sqlite
Call print_info(show_token=True) to show the Kleio Server token
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=test-project, project_home=/Users/jrc/develop/timelink-py/tests/timelink-home/projects/test-project, db_type=sqlite, db_name=rentitynb, kleio_image=timelinkserver/kleio-server, kleio_version=latest, postgres_image=po

In [5]:
from timelink.api.models import Person, REntity, Link
# log sql from sqlalchemy to stdout
import logging
logging.basicConfig()
logging.getLogger('sqlalchemy.engine').setLevel(logging.INFO)

with tlnb.db.session() as session:
    tlnb.db.drop_db(session=session)
    tlnb.db.create_tables()

with tlnb.db.session() as session:
    # Create a person
    person1 = Person(id='john-doe',name="John Doe", sex='m')
    person2 = Person(id='john-doe2',name="John Doe II", sex='m')
    session.add(person1)
    session.add(person2)
    session.commit()

with tlnb.db.session() as session:
    person1 = session.get(Person,'john-doe')
    person2 = session.get(Person,'john-doe2')
    print(person1)
    print(person2)


INFO  [sqlalchemy.engine.Engine] BEGIN (implicit)
INFO  [sqlalchemy.engine.Engine] SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
INFO  [sqlalchemy.engine.Engine] ROLLBACK
INFO  [sqlalchemy.engine.Engine] BEGIN (implicit)
INFO  [sqlalchemy.engine.Engine] SELECT classes.id 
FROM entities JOIN classes ON classes.id = entities.id
INFO  [sqlalchemy.engine.Engine] [cached since 282.2s ago] ()
INFO  [sqlalchemy.engine.Engine] INSERT INTO entities (id, class, updated) VALUES (?, ?, ?)
INFO  [sqlalchemy.engine.Engine] [cached since 183.9s ago] ('entity', 'class', '2024-11-05 07:29:18.511902')
INFO  [sqlalchemy.engine.Engine] INSERT INTO classes (id, table_name, group_name, super) VALUES (?, ?, ?, ?)
INFO  [sqlalchemy.engine.Engine] [cached since 183.9s ago] ('entity', 'entities', 'na', 'root')
INFO  [sqlalchemy.engine.Engine] INSERT INTO class_attributes (the_class, name, colname, colclass, 

person$John Doe/m/id=john-doe
person$John Doe II/m/id=john-doe2
